In [1]:
import sys
sys.path.append('..\\helpers')

import cv2
import random
import os
import matplotlib.pyplot as plt
import numpy as np
from loguru import logger
import pandas as pd
import seaborn as sns
from PIL import Image, ImageDraw
from functools import reduce
from tqdm import tqdm

from pywt import dwt2
import math
from scipy import ndimage


# From helpers directory
from preprocessing import Preprocessor
from morphological_sifter import MorphologicalSifter
from segmentation import Segmentation

import display


preprocessor = Preprocessor()
mms = MorphologicalSifter()
seg = Segmentation()

plt.rcParams['figure.figsize'] = (20, 18)
plt.style.use('classic')
plt.subplots_adjust(wspace=0, hspace=0)
sns.set()

# To allow auto reload to this notebook after modifying any external file imported
%load_ext autoreload
%autoreload 2

<Figure size 640x480 with 0 Axes>

In [2]:
# Set the paths to the positive images and groundtruth folders
positive_segmented_path = '../dataset/output/segmentation'

# Get a list of all the tif images in the positive images folder
positive_dirs = [f for f in os.listdir(positive_segmented_path) if f.endswith('.tif')]

positive_dirs

['20586908_6c613a14b80a8591_MG_R_CC_ANON_0.tif',
 '20586908_6c613a14b80a8591_MG_R_CC_ANON_1.tif',
 '20586908_6c613a14b80a8591_MG_R_CC_ANON_2.tif',
 '20586908_6c613a14b80a8591_MG_R_CC_ANON_3.tif',
 '20586934_6c613a14b80a8591_MG_L_CC_ANON_0.tif',
 '20586934_6c613a14b80a8591_MG_L_CC_ANON_1.tif',
 '20586934_6c613a14b80a8591_MG_L_CC_ANON_2.tif',
 '20586934_6c613a14b80a8591_MG_L_CC_ANON_3.tif',
 '20586960_6c613a14b80a8591_MG_R_ML_ANON_0.tif',
 '20586960_6c613a14b80a8591_MG_R_ML_ANON_1.tif',
 '20586960_6c613a14b80a8591_MG_R_ML_ANON_2.tif',
 '20586960_6c613a14b80a8591_MG_R_ML_ANON_3.tif',
 '20586986_6c613a14b80a8591_MG_L_ML_ANON_0.tif',
 '20586986_6c613a14b80a8591_MG_L_ML_ANON_1.tif',
 '20586986_6c613a14b80a8591_MG_L_ML_ANON_2.tif',
 '20586986_6c613a14b80a8591_MG_L_ML_ANON_3.tif',
 '20587612_f4b2d377f43ba0bd_MG_R_CC_ANON_0.tif',
 '20587612_f4b2d377f43ba0bd_MG_R_CC_ANON_1.tif',
 '20587612_f4b2d377f43ba0bd_MG_R_CC_ANON_2.tif',
 '20587612_f4b2d377f43ba0bd_MG_R_CC_ANON_3.tif',
 '20587664_f4b2d377f

In [3]:
for file_name in positive_dirs:
    semented_img_dir = os.path.join(positive_segmented_path, file_name)
    pv_img = cv2.imread(semented_img_dir, cv2.IMREAD_UNCHANGED)
    preprocessor.upsample(image = pv_img, export_upsampled = True, dir = semented_img_dir, type='segmentation')     

In [4]:
# Set the paths to the positive images and groundtruth folders
# groundtruth_path = '../dataset/processed/groundtruth'
groundtruth_path = '../dataset/processed/groundtruth'

# Get a list of all the tif images in the positive images folder
groundtruth_dirs = [f for f in os.listdir(groundtruth_path) if f.endswith('.tif')]

groundtruth_dirs

['20586908_6c613a14b80a8591_MG_R_CC_ANON.tif',
 '20586934_6c613a14b80a8591_MG_L_CC_ANON.tif',
 '20586960_6c613a14b80a8591_MG_R_ML_ANON.tif',
 '20586986_6c613a14b80a8591_MG_L_ML_ANON.tif',
 '20587612_f4b2d377f43ba0bd_MG_R_CC_ANON.tif',
 '20587664_f4b2d377f43ba0bd_MG_R_ML_ANON.tif',
 '20587758_81cd83d2f4d78528_MG_L_CC_ANON.tif',
 '20587810_81cd83d2f4d78528_MG_L_ML_ANON.tif',
 '20587902_8dbbd4e51f549ff0_MG_R_CC_ANON.tif',
 '20587928_8dbbd4e51f549ff0_MG_R_ML_ANON.tif',
 '20587994_024ee3569b2605dc_MG_R_CC_ANON.tif',
 '20588046_024ee3569b2605dc_MG_R_ML_ANON.tif',
 '20588190_8d0b9620c53c0268_MG_L_CC_ANON.tif',
 '20588216_8d0b9620c53c0268_MG_L_ML_ANON.tif',
 '20588308_493155e17143edef_MG_L_ML_ANON.tif',
 '20588334_493155e17143edef_MG_L_CC_ANON.tif',
 '20588536_bf1a6aaadb05e3df_MG_L_ML_ANON.tif',
 '20588562_bf1a6aaadb05e3df_MG_L_CC_ANON.tif',
 '20588680_036aff49b8ac84f0_MG_L_ML_ANON.tif',
 '22427705_d713ef5849f98b6c_MG_L_CC_ANON.tif',
 '22427751_d713ef5849f98b6c_MG_L_ML_ANON.tif',
 '22427840_bb

In [5]:
for file_name in groundtruth_dirs:
    gt_img_dir = os.path.join(groundtruth_path, file_name)
    pv_img = cv2.imread(gt_img_dir, cv2.IMREAD_GRAYSCALE)
    preprocessor.upsample(image = pv_img, export_upsampled = True, dir = gt_img_dir, type='groundtruth')     